## 安裝 kears-crontrib

In [8]:
! pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /private/var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/pip-t1_7py2z-build
  Running setup.py install for keras-contrib ... done
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## 讀取資料

In [1]:
from glob import glob
import numpy as np

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))
        #print(path)
        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('./datasets/%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('./datasets/%s/%sB/*' % (self.dataset_name, data_type))
        #print(path_A)
        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

## 讀取馬對斑馬轉換

In [2]:
a = DataLoader('horse2zebra')

## 設定網路參數

In [3]:
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os


# Input shape
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)

Using TensorFlow backend.


## 建立生成器

In [4]:
def build_generator():
    """U-Net Generator"""

    def conv2d(layer_input, filters, f_size=4):
        """Layers used during downsampling"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        d = InstanceNormalization()(d)
        return d

    def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
        """Layers used during upsampling"""
        u = UpSampling2D(size=2)(layer_input)
        u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
        if dropout_rate:
            u = Dropout(dropout_rate)(u)
        u = InstanceNormalization()(u)
        u = Concatenate()([u, skip_input])
        return u

    # Image input
    d0 = Input(shape=img_shape)

    # Downsampling
    d1 = conv2d(d0, gf)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)

    # Upsampling
    u1 = deconv2d(d4, d3, gf*4)
    u2 = deconv2d(u1, d2, gf*2)
    u3 = deconv2d(u2, d1, gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(d0, output_img)

## 建立鑑別器

In [5]:
def build_discriminator():

    def d_layer(layer_input, filters, f_size=4, normalization=True):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if normalization:
            d = InstanceNormalization()(d)
        return d

    img = Input(shape=img_shape)

    d1 = d_layer(img, df, normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity)

## 建立訓練過程

In [6]:
def train(epochs, batch_size=1, sample_interval=50):

    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    for epoch in range(epochs):
        for batch_i, (imgs_A, imgs_B) in enumerate(data_loader.load_batch(batch_size)):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Translate images to opposite domain
            fake_B = g_AB.predict(imgs_A)
            fake_A = g_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            dA_loss_real = d_A.train_on_batch(imgs_A, valid)
            dA_loss_fake = d_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

            dB_loss_real = d_B.train_on_batch(imgs_B, valid)
            dB_loss_fake = d_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)


            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = combined.train_on_batch([imgs_A, imgs_B],
                                                    [valid, valid,
                                                    imgs_A, imgs_B,
                                                    imgs_A, imgs_B])

            elapsed_time = datetime.datetime.now() - start_time

            # Plot the progress
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                    % ( epoch, epochs,
                                                                        batch_i, data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        np.mean(g_loss[1:3]),
                                                                        np.mean(g_loss[3:5]),
                                                                        np.mean(g_loss[5:6]),
                                                                        elapsed_time))

            # If at save interval => save generated image samples
            if batch_i % sample_interval == 0:
                sample_images(epoch, batch_i)

## 圖片取樣

In [7]:
def sample_images(epoch, batch_i):
    os.makedirs('images/%s' % dataset_name, exist_ok=True)
    r, c = 2, 3

    imgs_A = data_loader.load_data(domain="A", batch_size=1, is_testing=True)
    imgs_B = data_loader.load_data(domain="B", batch_size=1, is_testing=True)
    print(img_A)
    # Demo (for GIF)
    #imgs_A = data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
    #imgs_B = data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i, j].set_title(titles[j])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
    plt.close()

## 建構模型

In [8]:
# Configure data loader
dataset_name = 'horse2zebra'
data_loader = DataLoader(dataset_name=dataset_name,
                              img_res=(img_rows, img_cols))


# Calculate output shape of D (PatchGAN)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)

# Number of filters in the first layer of G and D
gf = 32
df = 64

# Loss weights
lambda_cycle = 10.0                    # Cycle-consistency loss
lambda_id = 0.1 * lambda_cycle    # Identity loss

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminators
d_A = build_discriminator()
d_B = build_discriminator()
d_A.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])
d_B.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

#-------------------------
# Construct Computational
#   Graph of Generators
#-------------------------

# Build the generators
g_AB = build_generator()
g_BA = build_generator()

# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

# Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)
# Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)
# Identity mapping of images
img_A_id = g_BA(img_A)
img_B_id = g_AB(img_B)

# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False

# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

# Combined model trains generators to fool discriminators
combined = Model(inputs=[img_A, img_B],
                      outputs=[ valid_A, valid_B,
                                reconstr_A, reconstr_B,
                                img_A_id, img_B_id ])
combined.compile(loss=['mse', 'mse',
                       'mae', 'mae',
                       'mae', 'mae'],
                    loss_weights=[  1, 1,
                                    lambda_cycle, lambda_cycle,
                                    lambda_id, lambda_id ],
                    optimizer=optimizer)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


## 開始訓練

In [ ]:
train(epochs=20, batch_size=10, sample_interval=10)

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/scipy/misc/pilutil.py:480: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/scipy/misc/pilutil.py:483: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/20] [Batch 0/31] [D loss: 30.473705, acc:  10%] [G loss: 48.588200, adv: 16.974632, recon: 0.647205, id: 0.800721] time: 0:00:28.406611 
Tensor("input_5:0", shape=(?, 128, 128, 3), dtype=float32)
[Epoch 0/20] [Batch 1/31] [D loss: 8.922375, acc:  15%] [G loss: 28.108395, adv: 7.612333, recon: 0.544633, id: 0.947274] time: 0:00:40.051470 
[Epoch 0/20] [Batch 2/31] [D loss: 14.264787, acc:   6%] [G loss: 112.850899, adv: 49.705875, recon: 0.576540, id: 0.816267] time: 0:00:51.601959 
[Epoch 0/20] [Batch 3/31] [D loss: 11.656817, acc:  13%] [G loss: 35.564556, adv: 11.348845, recon: 0.546520, id: 0.941505] time: 0:01:02.694095 
[Epoch 0/20] [Batch 4/31] [D loss: 11.041611, acc:  15%] [G loss: 14.775379, adv: 0.843940, recon: 0.558030, id: 0.954537] time: 0:01:13.907103 
[Epoch 0/20] [Batch 5/31] [D loss: 1.127777, acc:  29%] [G loss: 16.983524, adv: 1.649649, recon: 0.582375, id: 1.059489] time: 0:01:25.149874 
[Epoch 0/20] [Batch 6/31] [D loss: 0.772206, acc:  54%] [G loss: 13.8

[Epoch 1/20] [Batch 25/31] [D loss: 0.343576, acc:  50%] [G loss: 8.896963, adv: 1.105271, recon: 0.255960, id: 0.829949] time: 0:11:19.973290 
[Epoch 1/20] [Batch 26/31] [D loss: 0.450318, acc:  44%] [G loss: 7.964173, adv: 0.586459, recon: 0.255217, id: 0.816267] time: 0:11:31.793912 
[Epoch 1/20] [Batch 27/31] [D loss: 0.423925, acc:  34%] [G loss: 8.564905, adv: 0.792510, recon: 0.266553, id: 0.813380] time: 0:11:43.809087 
[Epoch 1/20] [Batch 28/31] [D loss: 0.575580, acc:  23%] [G loss: 7.732025, adv: 0.640258, recon: 0.246682, id: 0.726477] time: 0:11:55.755244 
[Epoch 1/20] [Batch 29/31] [D loss: 0.356946, acc:  46%] [G loss: 7.938466, adv: 0.858818, recon: 0.231849, id: 0.783448] time: 0:12:07.694241 
[Epoch 2/20] [Batch 0/31] [D loss: 0.311734, acc:  54%] [G loss: 8.261071, adv: 0.713998, recon: 0.258658, id: 0.843827] time: 0:12:19.428878 
Tensor("input_5:0", shape=(?, 128, 128, 3), dtype=float32)
[Epoch 2/20] [Batch 1/31] [D loss: 0.392425, acc:  36%] [G loss: 7.738689, adv

[Epoch 3/20] [Batch 20/31] [D loss: 0.395737, acc:  31%] [G loss: 7.209821, adv: 0.703001, recon: 0.212393, id: 0.827724] time: 0:22:20.132122 
Tensor("input_5:0", shape=(?, 128, 128, 3), dtype=float32)
[Epoch 3/20] [Batch 21/31] [D loss: 0.267652, acc:  56%] [G loss: 7.849110, adv: 0.891619, recon: 0.221161, id: 0.854983] time: 0:22:32.098434 
[Epoch 3/20] [Batch 22/31] [D loss: 0.365695, acc:  44%] [G loss: 7.239266, adv: 0.856749, recon: 0.203164, id: 0.692622] time: 0:22:43.845054 
[Epoch 3/20] [Batch 23/31] [D loss: 0.344024, acc:  54%] [G loss: 6.874855, adv: 0.676638, recon: 0.194480, id: 0.847280] time: 0:22:55.623637 
[Epoch 3/20] [Batch 24/31] [D loss: 0.264352, acc:  56%] [G loss: 7.372781, adv: 0.799683, recon: 0.210817, id: 0.747579] time: 0:23:07.308558 
[Epoch 3/20] [Batch 25/31] [D loss: 0.300429, acc:  54%] [G loss: 6.905420, adv: 0.728874, recon: 0.202249, id: 0.707569] time: 0:23:19.077238 
[Epoch 3/20] [Batch 26/31] [D loss: 0.343854, acc:  49%] [G loss: 7.885133, a